In [75]:
import sys
sys.path.append("../")
import os
import pandas as pd 
import numpy as np
import torch
import misc 
import SimpleITK as sitk
from config import model_config
import dataloader
from torch.utils.data import DataLoader

In [76]:
data = pd.read_csv("../dataset/ADNI/data_cleaned.csv")
data = data.set_index("PTID")
data

,RID,EXAMDATE,DX_bl,AGE,PTGENDER,PTEDUCAT,APOE4,CDRSB,ADAS13,MMSE,...,PET,DTI,Male,Female,APOE4_0,APOE4_1,APOE4_2,CN,MCI,AD
PTID,,,,,,,,,,,,,,,,,,,,,
011_S_0003,3,2005-09-12,AD,81.300000,Male,18,1.0,4.5,31.00,20.0,...,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
011_S_0003,3,2006-09-12,AD,82.300000,Male,18,1.0,3.5,35.00,17.0,...,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
011_S_0003,3,2007-09-12,AD,83.300000,Male,18,1.0,8.0,37.67,19.0,...,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
022_S_0004,4,2005-11-08,LMCI,67.500000,Male,10,0.0,1.0,21.33,27.0,...,0,0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
022_S_0004,4,2006-11-14,LMCI,68.516438,Male,10,0.0,1.0,22.00,26.0,...,0,0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
057_S_6869,6869,2022-03-01,AD,79.278082,Male,10,NaN,9.0,25.33,23.0,...,0,0,1.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0
057_S_6869,6869,2023-03-07,AD,80.294521,Male,10,NaN,11.0,33.67,22.0,...,0,0,1.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0
033_S_6889,6889,2020-10-23,LMCI,67.800000,Male,18,0.0,1.0,25.00,28.0,...,1,0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [77]:
# # Dataloaders:
# data = pd.read_csv("../dataset/ADNI/data_cleaned.csv")
# train_set = dataloader.MedDataset(path_root="G:/Dataset/ADNI/TrainingData", data_frame=data, config=model_config, mode="train")
# train_loader = DataLoader(train_set, batch_size=1, shuffle=True, num_workers=1, pin_memory=True)


In [79]:
patientInfo = data.loc["033_S_6889"]

In [80]:
FEATURES_COGNITIVE_SCORE = ["CDRSB", "ADAS13", "MMSE", "FAQ", "RAVLT_immediate", "RAVLT_learning", "RAVLT_perc_forgetting"]
FEATURES_DEMOGRAPHIC = ["PTEDUCAT", "AGE", "Male", "Female", "APOE4_0", "APOE4_1", "APOE4_2"]
patientInfo[FEATURES_COGNITIVE_SCORE + FEATURES_DEMOGRAPHIC]

,CDRSB,ADAS13,MMSE,FAQ,RAVLT_immediate,RAVLT_learning,RAVLT_perc_forgetting,PTEDUCAT,AGE,Male,Female,APOE4_0,APOE4_1,APOE4_2
PTID,,,,,,,,,,,,,,
033_S_6889,1.0,25.00,28.0,1.0,21.0,4.0,83.3333,18,67.800000,1.0,0.0,1.0,0.0,0.0
033_S_6889,1.5,23.33,27.0,1.0,28.0,5.0,100.0000,18,68.805479,1.0,0.0,1.0,0.0,0.0
033_S_6889,1.0,17.33,27.0,2.0,30.0,4.0,100.0000,18,69.769863,1.0,0.0,1.0,0.0,0.0


In [81]:
data_x = np.full([10 + 1, len(FEATURES_COGNITIVE_SCORE +FEATURES_DEMOGRAPHIC)], np.nan)
delta_x = np.zeros([10 + 1, len(FEATURES_COGNITIVE_SCORE +FEATURES_DEMOGRAPHIC)])
mask_x = np.zeros([10 + 1, len(FEATURES_COGNITIVE_SCORE +FEATURES_DEMOGRAPHIC)],  dtype=bool)

mask_i = np.zeros([11, 3], dtype=bool) 
delta_i = np.zeros([11, 256]) 

data_y = np.full([10 + 1, 3], np.nan)
delta_y = np.zeros([10 + 1, 3])
mask_y = np.zeros([10 + 1, 3],  dtype=bool)
for i in range(len(patientInfo)):
    year = int(patientInfo["Month"][i] // 12)
    data_x[year] = np.array(patientInfo[FEATURES_COGNITIVE_SCORE + FEATURES_DEMOGRAPHIC][patientInfo["Month"] == year*12])
    data_y[year] = np.array(patientInfo[["CN", "MCI", "AD"]][patientInfo["Month"] == year*12])

mask_x[data_x == data_x] = 1.
mask_y[data_y == data_y] = 1.

last_observed_x = np.zeros(len(FEATURES_COGNITIVE_SCORE) + len(FEATURES_DEMOGRAPHIC))
last_observed_y = np.zeros(3)
last_observed_i = np.zeros(256)
for tps in range( 10+ 1):
    delta_x[tps] = tps - last_observed_x
    last_observed_x[mask_x[tps]] = tps

    delta_y[tps] = tps - last_observed_y
    last_observed_y[mask_y[tps]] = tps
    

In [90]:
mask_i[0, 1] = 1
last_observed_i[np.any(mask_i[2]==True)] =1 
last_observed_i

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [95]:
np.any(mask_i[0]==True)

True

In [133]:
torch.sin(torch.zeros(1)).cuda()

AssertionError: Torch not compiled with CUDA enabled

In [25]:
path_to_nifti = "G:/Dataset/ADNI/3_smooth (SPM)/002_S_0295/sw002_S_0295_Month_062.6_PET_AV45_2011-06-10.nii"
image1 = sitk.GetArrayFromImage(sitk.ReadImage(str(path_to_nifti)))

path_to_nifti = "G:/Dataset/ADNI/TrainingData/002_S_0295/visit_060_PET_AV45.nii"
image2 = sitk.GetArrayFromImage(sitk.ReadImage(str(path_to_nifti)))
(image1-image2).sum()

0.0

In [ ]:
image

In [ ]:
np.pad(image, ((1,0), (1,0), (1, 0)), 'constant', constant_values=0).shape

In [62]:
mu = np.ones((3, 2, 3))

T = np.zeros((3, 2, 3))
mask = np.ones((3, 2), dtype=bool)
(mu * T)[mask].shape

IndexError: boolean index did not match indexed array along dimension 2; dimension is 3 but corresponding boolean dimension is 1

In [63]:
mu[0, 0] =10
mu[1, 0] =5
mu[0, 1] =2
mu

array([[[10., 10., 10.],
        [ 2.,  2.,  2.]],

       [[ 5.,  5.,  5.],
        [ 1.,  1.,  1.]],

       [[ 1.,  1.,  1.],
        [ 1.,  1.,  1.]]])

In [64]:
mask[0, 0] =0
mask[0, 1] =0
mask

array([[[False],
        [False]],

       [[ True],
        [ True]],

       [[ True],
        [ True]]])

In [70]:
y_t_hat_seq  = torch.empty((2, 5, 3))
y_t_hat = torch.ones(2, 3)

In [71]:
y_t_hat_seq[:, 0, :] = y_t_hat
y_t_hat_seq

tensor([[[1.0000e+00, 1.0000e+00, 1.0000e+00],
         [6.7262e-43, 8.3162e-29, 6.7262e-43],
         [1.4013e-45, 0.0000e+00, 4.2039e-45],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[1.0000e+00, 1.0000e+00, 1.0000e+00],
         [1.4013e-45, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 3.3631e-44, 0.0000e+00]]])